# Get Monthly Prices for Apple

This notebook fetches end-of-month adjusted closing prices for Apple (AAPL) from the Rice Data Portal.

In [ ]:
from rice_data_client import RiceDataClient
import pandas as pd
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Get access token from environment
ACCESS_TOKEN = os.getenv('RICE_ACCESS_TOKEN')

if not ACCESS_TOKEN:
    raise ValueError(
        "RICE_ACCESS_TOKEN not found in .env file. \n"
        "Please create a .env file with your access token from data-portal.rice-business.org:\n"
        "RICE_ACCESS_TOKEN=your_access_token_here"
    )

# Initialize client
client = RiceDataClient(access_token=ACCESS_TOKEN)

## Strategy: Loop Over Years to Get Daily Prices

Following the rice-data-query skill guidance, we'll:
1. Loop through years from 2020 to current
2. Download daily prices for each year
3. Filter to end-of-month dates using pandas
4. Concatenate all years

This avoids server timeouts for large queries.

In [ ]:
from datetime import datetime

# Get current year
current_year = datetime.now().year
start_year = 2020

# Store month-end data for each year
monthly_data = []

for year in range(start_year, current_year + 1):
    print(f"Fetching data for {year}...")
    
    # Query daily prices for this year
    sql = f"""
    SELECT ticker, date::DATE as date, closeadj
    FROM sep
    WHERE ticker = 'AAPL'
      AND date::DATE >= '{year}-01-01'
      AND date::DATE < '{year + 1}-01-01'
    ORDER BY date
    """
    
    df_year = client.query(sql)
    
    if len(df_year) == 0:
        continue
    
    # Convert date to datetime
    df_year['date'] = pd.to_datetime(df_year['date'])
    
    # Create year-month column
    df_year['year_month'] = df_year['date'].dt.to_period('M')
    
    # Get the last trading day of each month
    df_month_end = df_year.groupby(['ticker', 'year_month']).apply(
        lambda x: x.loc[x['date'].idxmax()]
    ).reset_index(drop=True)
    
    # Drop the helper column
    df_month_end = df_month_end.drop('year_month', axis=1)
    
    monthly_data.append(df_month_end)
    print(f"  Found {len(df_month_end)} month-ends")

# Concatenate all years
df = pd.concat(monthly_data, ignore_index=True)
df = df.sort_values('date').reset_index(drop=True)

print(f"\nTotal monthly observations: {len(df)}")
print(f"Date range: {df['date'].min()} to {df['date'].max()}")

In [ ]:
# Preview the data
df.head(10)

In [ ]:
# Preview the most recent data
df.tail(10)

## Save Data as Parquet File

Save this data locally so you can use it in future sessions without re-querying the database.

In [ ]:
# What filename would you like to use to save this data?
filename = "aapl_monthly_prices.parquet"

df.to_parquet(filename)
print(f"Data saved to {filename} ({len(df)} rows)")

## Plot Monthly Prices

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(df['date'], df['closeadj'], linewidth=2)
plt.title('Apple (AAPL) Monthly Adjusted Closing Prices', fontsize=14, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Adjusted Close Price ($)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()